<a href="https://colab.research.google.com/github/goal2002man/USD_Computer_Vision_Final/blob/bryan/LP_detect_and_char_segment_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import urllib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os
import time
import PIL
from PIL import Image

from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Input, Dropout
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [2]:
import torch
from google.colab.patches import cv2_imshow
from imutils import perspective
import random

In [3]:
import functools
import re
import cvxpy as cp

In [4]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 14474, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 14474 (delta 65), reused 67 (delta 42), pack-reused 14371
Receiving objects: 100% (14474/14474), 13.60 MiB | 25.74 MiB/s, done.
Resolving deltas: 100% (9966/9966), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 4.7 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 1.6 MB 61.1 MB/s 


In [5]:
!git clone https://github.com/goal2002man/USD_Computer_Vision_Final/

Cloning into 'USD_Computer_Vision_Final'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 67 (delta 25), reused 34 (delta 11), pack-reused 0
Unpacking objects: 100% (67/67), done.


In [6]:
!git clone -b model_weights https://github.com/goal2002man/USD_Computer_Vision_Final/

fatal: destination path 'USD_Computer_Vision_Final' already exists and is not an empty directory.


In [7]:
!ls USD_Computer_Vision_Final

best_char.pt	       LP_detect_and_char_segment_v1.ipynb
best_lp.pt	       README.md
character_recognition  yolov5
environment.yml        YOLOv5_License_Plate_Detector.ipynb


In [8]:
# License Plate Detector
path_yolo_best_weights = '/content/USD_Computer_Vision_Final/best_lp.pt'

lpd_model = torch.hub.load('yolov5/', 'custom', path=path_yolo_best_weights, source='local')  # local repo

YOLOv5 🚀 v7.0-32-g357cde9 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [9]:
# Charachter Segmentation
path_yolo_best_weights_char_segmenter = '/content/USD_Computer_Vision_Final/best_char.pt' 

char_segm_model = torch.hub.load('yolov5/', 'custom', path=path_yolo_best_weights_char_segmenter, source='local')  # local repo

YOLOv5 🚀 v7.0-32-g357cde9 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [47]:
def char_segmentation_yolo(file_name):
 input_images = cv2.imread(file_name)
 results = lpd_model(input_images)
 boxes = results.pandas().xyxy[0]  
 ###cv2_imshow(input_images)

 # Identify index of license plate bounding box
 index_lp = 10000
 flag = 0
 for i in range(boxes.shape[0]):
   if boxes.loc[i].at["name"] == 'license-plate':
     index_lp = i
     flag = 1
     break
 if flag == 0:
     display ('No license plate detected for file: ' + str(file_name))
     return None, None, None, None

 # Coordinates of bounding box with highest confidence interval
 bb_xmin = boxes.loc[index_lp].at["xmin"]
 bb_xmax = boxes.loc[index_lp].at["xmax"]
 bb_ymin = boxes.loc[index_lp].at["ymin"]
 bb_ymax = boxes.loc[index_lp].at["ymax"]

 # # Image patch for character segmentation
 img_for_char_segm = input_images[round(bb_ymin):round(bb_ymax), round(bb_xmin):round(bb_xmax)]
 ###cv2_imshow(img_for_char_segm)

 # Charachter segmentation
 results_segm = char_segm_model(img_for_char_segm)
 boxes_segm = results_segm.pandas().xyxy[0]
 ###print(boxes_segm)
 
 ## Display bounding boxes for characters and add the cropped charachters to a list
 char_images = []
 img_for_char_segm_copy = img_for_char_segm.copy()
 for i in range(boxes_segm.shape[0]):
   if boxes_segm.loc[i].at["name"] == 'character':
     bb_segm_xmin = boxes_segm.loc[i].at["xmin"]
     bb_segm_xmax = boxes_segm.loc[i].at["xmax"]
     bb_segm_ymin = boxes_segm.loc[i].at["ymin"]
     bb_segm_ymax = boxes_segm.loc[i].at["ymax"]
     crop_char = img_for_char_segm[round(bb_segm_ymin):round(bb_segm_ymax), round(bb_segm_xmin):round(bb_segm_xmax)]
     ###cv2_imshow(cv2.resize(crop_char,(100,100)))
     char_images.append(crop_char)
     cv2.rectangle(img_for_char_segm_copy, (int(bb_segm_xmin),int(bb_segm_ymin)), (int(bb_segm_xmax),int(bb_segm_ymax)), (255,0,0), 1)

 ###cv2_imshow(cv2.resize(img_for_char_segm_copy,(300,100)))

 return input_images, img_for_char_segm, char_images, boxes_segm

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
list_of_images = os.listdir(
    '/content/drive/MyDrive/Intro to Computer Vision - Final/data/train/images') + os.listdir(
    '/content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images'
)

len(list_of_images)

4392

In [51]:
# Call the function, looping over all files and recording the DF results - first for the Training data
output_df = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class', 'name'])

for f in list_of_images:
  filename = '/content/drive/MyDrive/Intro to Computer Vision - Final/data/train/images/' + f  # UPDATE with your file name
  input_images, img_for_char_segm, char_images , boxes_df=  char_segmentation_yolo(filename)
  if boxes_df is not None:
    boxes_df = boxes_df.assign(filename = f)

  output_df = output_df.append(boxes_df)


'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/train/images/track0058[19].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/train/images/track0058[24].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/train/images/track0058[18].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/train/images/track0058[20].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/train/images/track0058[27].png'

...

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/train/images/track0011[10].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/train/images/track0011[11].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/train/images/track0011[09].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/train/images/track0020[24].png'

FileNotFoundError: ignored

In [55]:
# Call the function, looping over all files and recording the DF results -- this time for the Test portion
test_output_df = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class', 'name'])

list_of_test_images =  os.listdir(
    '/content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images'
)

for f in list_of_test_images:
  filename = '/content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images/' + f  # UPDATE with your file name
  input_images, img_for_char_segm, char_images , boxes_df=  char_segmentation_yolo(filename)
  if boxes_df is not None:
    boxes_df = boxes_df.assign(filename = f)

  test_output_df = test_output_df.append(boxes_df)

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images/track0115[27].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images/track0115[30].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images/track0115[28].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images/track0115[29].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images/track0118[17].png'

...

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images/track0114[28].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images/track0115[18].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images/track0115[16].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images/track0115[20].png'

'No license plate detected for file: /content/drive/MyDrive/Intro to Computer Vision - Final/data/test/images/track0115[15].png'

In [53]:
train_output_df = output_df

In [54]:
train_output_df

,xmin,ymin,xmax,ymax,confidence,class,name,filename
0,17.102282,6.640908,31.740005,25.190994,0.475688,0,character,track0056[15].png
0,90.010750,9.700005,103.706497,29.007738,0.369868,0,character,track0056[16].png
1,16.880816,6.687716,32.777382,24.927717,0.252908,0,character,track0056[16].png
0,16.663185,10.627225,30.272718,29.297670,0.585876,0,character,track0056[14].png
1,28.774498,10.950685,44.934467,30.779533,0.511155,0,character,track0056[14].png
...,...,...,...,...,...,...,...,...
1,83.052620,8.495237,93.463203,25.330498,0.878152,0,character,track0022[21].png
2,7.435962,8.989074,18.238316,25.144522,0.867433,0,character,track0022[21].png
3,47.780598,7.684345,58.095089,24.126379,0.867180,0,character,track0022[21].png
4,71.264000,8.339413,82.076881,24.188679,0.845991,0,character,track0022[21].png


In [56]:
test_output_df

,xmin,ymin,xmax,ymax,confidence,class,name,filename
0,63.847225,9.963105,77.744057,28.803953,0.902511,0,character,track0116[01].png
1,89.697945,8.976999,103.343758,27.516697,0.886228,0,character,track0116[01].png
2,76.618568,9.253776,90.597931,27.991692,0.882751,0,character,track0116[01].png
3,6.358106,11.416066,20.251127,31.874117,0.880720,0,character,track0116[01].png
4,31.877413,11.276544,45.997524,31.189083,0.876594,0,character,track0116[01].png
...,...,...,...,...,...,...,...,...
2,21.825329,14.576013,41.055138,41.230034,0.871493,0,character,track0115[10].png
3,39.038902,15.922177,57.903160,42.832375,0.871132,0,character,track0115[10].png
4,99.397026,20.206547,117.972969,46.293152,0.861077,0,character,track0115[10].png
5,66.567711,16.791142,81.921921,45.165394,0.799420,0,character,track0115[10].png
